## Task: Find most expensive area in Bavaria 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

1. Einlesen der Daten aus gegebener Tabelle
    * df ist der main DataFrame mit den Preisen je nach Gemeinde

In [ ]:
df = pd.read_excel('Data/Kaufpreis.xlsx')
df = df.iloc[6:, :]
df.columns = ['ID', 'Gemeinde', 'Bauland_Anzahl', 'Baureifes_Anzahl', 'BaulandQM', 'BaureifQM', 'BaulandEuro', 'BaureifEuro', 'BaulandEQM', 'BaureifEQM']

2. Lösche aus Tabelle alle leeren Zeilen sowie Einträge mit weniger als 10 Einträgen (Verkäufen)

In [ ]:
# Lösche leere Zeielen aus Tabelle
df['Bauland_Anzahl'].replace('-', np.nan, inplace=True)
df.dropna(subset = ['Bauland_Anzahl'], inplace=True)
# Lösche Gemeinden mit weniger als 10 Einträgen
df = df[~(df['Bauland_Anzahl'] <= 10)]
print(df)

3. Erstellen eines neuen Dataframes nur mit benötigten Spalten
    * Es werden dabei zwei (optisch) identische neue Dataframes angelegt
        * collection: Zusammengeführte Tabelle, mit ID als String (für weitere Verwendung)
        * collectionInt: ID als INT, um nach Gemeinden, Landkreisen und Regierungsbezirken einfach filtern zu können

In [ ]:
gemeinde = df['Gemeinde'].astype(str)
collection = pd.DataFrame(gemeinde)

#baulandAnz = df['Bauland_Anzahl'].astype(float)
#collection['Bauland_Anzahl'] = baulandAnz

baulandEQM = df['BaulandEQM'].astype(float)
collection['BaulandEQM'] = baulandEQM

ID = df['ID']
collection['ID'] = ID

4. Ausgabe der Liste ohne Städte (nur Landkreise) sortiert nach teurstem Landkreis
    --> Siehe Erste Visualisierung

5. Laden einer neuen Tabelle (df2) mit Informationen über die Flächennutzung in den einzelnen Gemeinden
    * Zusammenführen der gesammelten Informationene in eine neue Tabelle results

In [ ]:
df2 = pd.read_excel('Data/AreaUsage.xlsx')
df2 = df2.iloc[6:, :]
df2.columns = ['ID', 'Gemeinde2', 'FlächeGesamt', 'Siedlung', 'Verkehr', 'Vegetation', 'Gewässer']
df2.replace('-', np.nan, inplace=True)
df2.dropna(inplace=True)
result = pd.merge(collection, df2, left_on='ID', right_on='ID', how='left').drop('Gemeinde2', axis=1)

6. Alle Werte müssen als numerischer Wert vorliegen. Sie werden deshalb vorsorglich umgeformt

In [ ]:
pd.to_numeric(result['FlächeGesamt']);
pd.to_numeric(result['Siedlung']);
pd.to_numeric(result['Verkehr']);
pd.to_numeric(result['Vegetation']);
pd.to_numeric(result['Gewässer']);

7. Anschließend können die Ergebnisse in einer eigenen Tabelle zwischengespeichert werden

In [ ]:
#result.to_excel("Data/Collection.xlsx")
#print(result)

In [ ]:
correlation = result[['BaulandEQM', 'Siedlung']]
corr_df = result.corr(method='spearman')
#corr_df = correlation.corr(method='spearman')
corr_df.style.background_gradient(cmap='coolwarm')
#print("The correlation DataFrame is:")
print(corr_df, "\n")

In [ ]:
import seaborn as sns
corr = result.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

### Ergebnis aus Flächennutzung: 
Es besteht eine leichte Korellation zwischen dem Preis pro QM Bauland und der Siedlungsfläche (0.224403) 

8. Hinzufügen einer neuen Tabelle mit Informationen zu dem Pendlersaldo in die einzelnen Gemeinden
    * Diese müssen ebenfalls zunächst gefiltert, angepasst und neu definiert werden

In [ ]:
df3 = pd.read_excel('Data/Pendlersaldo.xlsx', sheet_name="Gemeinden")
df3 = df3.iloc[8:, :]
df3.columns = ['ID', 'Gemeinde', 'Puffer', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'Pendlersaldo']
df3['Pendlersaldo'].replace('-', np.nan, inplace=True)
df3.dropna(subset = ['Pendlersaldo'], inplace=True)
print(df3)

9. Anlegen einer neuen Tabelle nur mit Pendlerinformationen. Da ID nicht einheitlich, muss stattdessen über den Namen der Gemeinde verknüpft werden
    * https://www.statistikdaten.bayern.de/genesis/online?operation=ergebnistabelleUmfang&levelindex=2&levelid=1647274508995&downloadname=13111-101r#abreadcrumb

In [ ]:
gemeinde = df3['Gemeinde'].astype(str)
pendler = pd.DataFrame(gemeinde)
saldo = df3['Pendlersaldo']
pendler['Pendlersaldo'] = saldo
print(pendler.sort_values(by=['Gemeinde'])[0:20])

In [ ]:
result.Gemeinde = result.Gemeinde.str.encode('utf-8')
pendler.Gemeinde = pendler.Gemeinde.str.encode('utf-8')
print(result.dtypes)
print('pass')
print(pendler.dtypes)

In [ ]:
result.Gemeinde = result.Gemeinde.str.strip()
pendler.Gemeinde = pendler.Gemeinde.str.strip()

In [ ]:
result.reset_index(drop=True)
result2 = pd.merge(result, pendler, left_on='Gemeinde', right_on='Gemeinde', how='left')#.drop('Gemeinde2', axis=1)
print(result2.sort_values(by=['Gemeinde']))

In [ ]:
#result2.to_excel("Data/PendlerSammlung.xlsx")

In [ ]:
result2['Pendlersaldo'].replace('•', np.nan, inplace=True)
result2.dropna(subset = ['Pendlersaldo'], inplace=True)

#result2.replace('.', np.nan, inplace=True)
#result2.dropna(inplace=True)
print(result2[345:350])

In [ ]:
#pd.to_numeric(result2['Pendlersaldo']);
corr_df = result2.corr(method='spearman')
print(corr_df)

### Ergebnis aus Pendlersaldo:
   * Es besteht (nahezu) keine Korrellation (-0.067196) zwischen dem Pendlersaldo und den Grundstückswerten 
   

In [ ]:
corr = result2.corr()
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)